# Vacan text preprocessing

In [2]:
import pandas as pd
import bs4
import re
from IPython.display import display

In [3]:
vacancies = pd.read_csv('data/vacancies.csv', header=0, sep='|')

In [4]:
vacancies.html = vacancies.html.astype(str)

In [5]:
vacancies.date.max()

'2015-12-06 12:39:17'

In [6]:
soup = bs4.BeautifulSoup(vacancies.iloc[0].html, 'html.parser')

In [7]:
soup.meta

<meta content="RUR" itemprop="salaryCurrency"><meta content="70000" itemprop="baseSalary">от 70 000 руб.</meta></meta>

# Get salary / text

In [8]:
def get_salary(html: str) -> tuple:
    soup = bs4.BeautifulSoup(html, 'html.parser')
    basesalary = None
    salarycurrency = None
    for meta in soup.findAll("meta"):
        meta_name = meta.get('itemprop', '').lower()
        if meta_name == 'salarycurrency':
            salarycurrency = meta.get('content', '').lower()
        if meta_name == 'basesalary':
            basesalary = int(meta.get('content', ''))
    return salarycurrency, basesalary

def get_text(html: str) -> str:
    try:
        x = re.search(r'</table>.*</div>(.*)Тип занятости', html)
        return x.groups()[0].lower()
    except AttributeError:
        return html.lower()

In [9]:
vacancies['text'] = vacancies.html.apply(get_text)

In [10]:
salaries = vacancies.html.apply(get_salary)

In [11]:
vacancies['salary'] = salaries.apply(lambda x: x[1])

In [12]:
vacancies['salary_currency'] = salaries.apply(lambda x: x[0])

In [13]:
vacancies['name'] = vacancies.name.apply(lambda x: x.lower())

In [14]:
vacancies.head(2)

,id,name,html,url,site,date,text,salary,salary_currency
0,1413553876,разработчик ruby/ruby on rails,"<div class=""b-important b-vacancy-info""><table...",http://irkutsk.hh.ru/vacancy/11833695?query=%D...,hh.ru,2014-10-17 13:51:12,"в состав распределенной команды, разрабатывающ...",70000,rur
1,1413553877,ведущий программист 1 с/руководитель направлен...,"<div class=""b-important b-vacancy-info""><table...",http://irkutsk.hh.ru/vacancy/11926858?query=%D...,hh.ru,2014-10-17 13:51:12,медицинская сеть клиник приглашает на работу н...,110000,rur


In [15]:
vacancies_prepared = vacancies[['date', 'name', 'text', 'salary', 'salary_currency', 'url']]

In [16]:
vacancies_prepared['currency'] = vacancies_prepared.loc[:,'salary_currency']

/usr/local/lib/python3.4/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [17]:
del vacancies_prepared['salary_currency']

In [18]:
vacancies_prepared.head()

,date,name,text,salary,url,currency
0,2014-10-17 13:51:12,разработчик ruby/ruby on rails,"в состав распределенной команды, разрабатывающ...",70000,http://irkutsk.hh.ru/vacancy/11833695?query=%D...,rur
1,2014-10-17 13:51:12,ведущий программист 1 с/руководитель направлен...,медицинская сеть клиник приглашает на работу н...,110000,http://irkutsk.hh.ru/vacancy/11926858?query=%D...,rur
2,2014-10-17 13:51:12,java-разработчик,обязанности: участие в проектах компании по р...,40000,http://irkutsk.hh.ru/vacancy/11968661?query=%D...,rur
3,2014-10-17 13:51:12,ios разработчик,"привет. меня зовут егор, в 2011 году мы создал...",120000,http://irkutsk.hh.ru/vacancy/11799508?query=%D...,rur
4,2014-10-17 13:51:12,программист php (web-разработка; back-end),в телекоммуникационную компанию требуется прог...,80000,http://irkutsk.hh.ru/vacancy/11639762?query=%D...,rur


In [19]:
vacancies_prepared.to_csv('data/vacancies_prepared.csv', index=True, sep='|')

# Tokenizing

In [20]:
from nltk.corpus import stopwords
stop_words = stopwords.words('russian')
stop_words.extend(['что', 'это', 'так', 'вот', 'быть', 'как', 'в', '—', 'к', 'на'])
stop_words.extend(['разработчик', 'программист', 'developer', 'разработка'])  # add custom stopwords
stop_words = set(stop_words)

In [21]:
import string
punctuation = set(string.punctuation)

In [22]:
from nltk.tokenize import RegexpTokenizer
import nltk
tokenizer = RegexpTokenizer(r'\w+')

In [23]:
def tokenize(text: str) -> list:
    text = text.replace('/', ' ').replace('-', ' ')
    tokens = nltk.word_tokenize(text)
    tokens = [i for i in tokens if i not in stop_words and i not in punctuation]
    tokens = [i.replace("«", "").replace("»", "") for i in tokens]
    return tokens

In [24]:
vacancies_tokens = pd.DataFrame()

In [25]:
vacancies_tokens['name_tokens'] = vacancies_prepared.name.apply(tokenize)

In [26]:
vacancies_tokens['text_tokens'] = vacancies_prepared.text.apply(tokenize)

In [27]:
vacancies_tokens.head()

,name_tokens,text_tokens
0,"[ruby, ruby, on, rails]","[состав, распределенной, команды, разрабатываю..."
1,"[ведущий, 1, руководитель, направления, it]","[медицинская, сеть, клиник, приглашает, работу..."
2,[java],"[обязанности, участие, проектах, компании, раз..."
3,[ios],"[привет, зовут, егор, 2011, году, создали, ком..."
4,"[php, web, back, end]","[телекоммуникационную, компанию, требуется, ph..."


In [28]:
back = vacancies_tokens.copy()

## Stemming

In [29]:
stemmer = nltk.stem.snowball.RussianStemmer(ignore_stopwords=True)

In [30]:
english_letters = set(string.ascii_lowercase)

In [31]:
def stem(tokens: list) -> list:
    return [stemmer.stem(token) if len(token) > 0 and token[0] not in english_letters else token for token in tokens]

In [32]:
vacancies_tokens['name_tokens'] = vacancies_tokens.name_tokens.apply(stem)

In [33]:
vacancies_tokens.head()

,name_tokens,text_tokens
0,"[ruby, ruby, on, rails]","[состав, распределенной, команды, разрабатываю..."
1,"[ведущ, 1, руководител, направлен, it]","[медицинская, сеть, клиник, приглашает, работу..."
2,[java],"[обязанности, участие, проектах, компании, раз..."
3,[ios],"[привет, зовут, егор, 2011, году, создали, ком..."
4,"[php, web, back, end]","[телекоммуникационную, компанию, требуется, ph..."


In [34]:
vacancies_tokens['text_tokens'] = vacancies_tokens.text_tokens.apply(stem)

In [35]:
vacancies_tokens_back = vacancies_tokens.copy()

# Got processed texts

In [36]:
vacancies = vacancies_tokens.join(vacancies_prepared)# [['date', 'salary', 'currency']])

In [41]:
def list_to_str(lst):
    return ' '.join(lst)

In [42]:
vacancies['name_tokens'] = vacancies.name_tokens.apply(list_to_str)
vacancies['text_tokens'] = vacancies.text_tokens.apply(list_to_str)

In [43]:
vacancies.head(20)

,name_tokens,text_tokens,date,name,text,salary,url,currency
0,ruby ruby on rails,соста распределен команд разрабатыва сист брон...,2014-10-17 13:51:12,разработчик ruby/ruby on rails,"в состав распределенной команды, разрабатывающ...",70000,http://irkutsk.hh.ru/vacancy/11833695?query=%D...,rur
1,ведущ 1 руководител направлен it,медицинск сет клиник приглаша работ работ веду...,2014-10-17 13:51:12,ведущий программист 1 с/руководитель направлен...,медицинская сеть клиник приглашает на работу н...,110000,http://irkutsk.hh.ru/vacancy/11926858?query=%D...,rur
2,java,обязан участ проект компан разработк аналитиче...,2014-10-17 13:51:12,java-разработчик,обязанности: участие в проектах компании по р...,40000,http://irkutsk.hh.ru/vacancy/11968661?query=%D...,rur
3,ios,привет зовут егор 2011 год созда компан gettab...,2014-10-17 13:51:12,ios разработчик,"привет. меня зовут егор, в 2011 году мы создал...",120000,http://irkutsk.hh.ru/vacancy/11799508?query=%D...,rur
4,php web back end,телекоммуникацион компан треб php постоя основ...,2014-10-17 13:51:12,программист php (web-разработка; back-end),в телекоммуникационную компанию требуется прог...,80000,http://irkutsk.hh.ru/vacancy/11639762?query=%D...,rur
5,1с,компан эспокад лидер рынк домашн текстил предл...,2014-10-17 13:51:12,программист-разработчик 1с,компания «эспокада» (один из лидеров рынка дом...,100000,http://irkutsk.hh.ru/vacancy/11869510?query=%D...,rur
6,,обязан написан нов программ автоматизац работ ...,2014-10-17 13:51:12,программист,обязанности: написание новых программ для авт...,50000,http://irkutsk.hh.ru/vacancy/11988450?query=%D...,rur
7,инженер машин обучен искусствен интеллект,обязан эффективн реализац алгоритм машин обуче...,2014-10-17 13:51:12,инженер-программист (машинное обучение и искус...,обязанности: эффективная реализация алгоритмо...,80000,http://irkutsk.hh.ru/vacancy/11803952?query=%D...,rur
8,1с 8.2 системн администратор,1с 8.2 системн администратор обязан программир...,2014-10-17 13:51:12,программист 1с ( 8.2) / системный администратор,программист 1с ( 8.2) / + системный администра...,80000,http://irkutsk.hh.ru/vacancy/11846449?query=%D...,rur
9,python,обязан серверн част умн дом интеграц различн у...,2014-10-17 13:51:12,python разработчик,обязанности: разработка серверной части умног...,50000,http://irkutsk.hh.ru/vacancy/12001783?query=%D...,rur


In [44]:
vacancies.to_csv('data/vacancies_stemmed.csv', sep='|', index=False)

In [45]:
vacancies_back = vacancies.copy()

# TF-Idf features

In [56]:
vacancies = vacancies_back.copy()

In [57]:
vacancies.name_tokens = vacancies.name_tokens.apply(lambda lst: ' '.join(lst))  # create texts
vacancies.text_tokens = vacancies.text_tokens.apply(lambda lst: ' '.join(lst))  # create texts

In [58]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [59]:
vectorizer = TfidfVectorizer(min_df=1)
text_tfidf_matrix = vectorizer.fit_transform(vacancies.name_tokens)

In [64]:
text_tfidf_matrix

<17446x1931 sparse matrix of type '<class 'numpy.float64'>'
	with 40945 stored elements in Compressed Sparse Row format>

In [65]:
vectorizer = TfidfVectorizer(min_df=1)
title_tfidf_matrix = vectorizer.fit_transform(vacancies.text_tokens)

In [66]:
title_tfidf_matrix

<17446x37282 sparse matrix of type '<class 'numpy.float64'>'
	with 1888943 stored elements in Compressed Sparse Row format>

# Very basic linear regression model

see: http://scikit-learn.org/stable/auto_examples/linear_model/plot_ols.html

In [63]:
X = text_tfidf_matrix.todense()
Y = vacancies.salary

In [67]:
from sklearn.preprocessing import Imputer
X = Imputer().fit_transform(X)

In [ ]:
# TODO: Split the data into training/testing sets